# Esempio Base di AutoGen

In questo esempio di codice, utilizzerai il framework AI [AutoGen](https://aka.ms/ai-agents/autogen) per creare un agente di base.

L'obiettivo di questo esempio è mostrarti i passaggi che utilizzeremo successivamente negli altri esempi di codice per implementare i diversi schemi agentici.


## Importa i Pacchetti Python Necessari


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Creare il Client

In questo esempio, utilizzeremo [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) per accedere al LLM.

Il `model` è definito come `gpt-4o-mini`. Prova a cambiare il modello con un altro disponibile nel marketplace di GitHub Models per vedere risultati diversi.

Come test rapido, eseguiremo semplicemente un prompt - `Qual è la capitale della Francia`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definire l'Agente

Ora che abbiamo configurato il `client` e confermato che funziona, creiamo un `AssistantAgent`. Ogni agente può essere assegnato a:  
**name** - Un nome breve che sarà utile per fare riferimento ad esso nei flussi multi-agente.  
**model_client** - Il client che hai creato nel passaggio precedente.  
**tools** - Strumenti disponibili che l'Agente può utilizzare per completare un compito.  
**system_message** - Il metaprompt che definisce il compito, il comportamento e il tono del LLM.  

Puoi modificare il system message per vedere come risponde il LLM. Tratteremo i `tools` nella Lezione #4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Esegui l'Agente

La funzione seguente eseguirà l'agente. Utilizziamo il metodo `on_message` per aggiornare lo stato dell'Agente con il nuovo messaggio.

In questo caso, aggiorniamo lo stato con un nuovo messaggio dell'utente che è `"Organizzami una fantastica vacanza al sole"`.

Puoi modificare il contenuto del messaggio per vedere come il LLM risponde in modo diverso.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
